In [1]:
# GET stock data

In [2]:
import json
import requests
import pandas as pd
import time
from fake_useragent import UserAgent
from datetime import datetime
from sqlalchemy import create_engine
from collections import Counter
import mysql.connector
import setting
import csv

In [3]:
def mysql_to_df():
    mydb = mysql.connector.connect(
            host="192.168.22.110",
            user="root",
            passwd="Pn123456",
            database="Stock"
    )

    mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM stock_list")
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult,columns=["stock_number","stock_name","stock_createdate","stock_class","stock_type"])
    
    mycursor.close()
    mydb.close()
    
    return df


In [4]:
df = mysql_to_df()
stock_list = []
for i in df["stock_number"]:
    stock_list.append(str(i))

In [5]:

# stock = "2330"
# # local_date = datetime.now().strftime('%Y%m%d')
# local_date = "20200101"
# timestamp = str(int(time.time()))
# url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp


In [6]:
def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d

In [7]:
def Get_stock_data(url):
    
    ua = UserAgent()
    headers = {"User-Agent":ua.random}
    res = requests.get(url,headers=headers,timeout=10.0)
    json_stock = json.loads(res.content)
    print("status_code: ",res.status_code)
    result = []
    
    for i in range(len(json_stock["data"])):
        try:
            Date = datetime.strptime(transform_date(json_stock["data"][i][0]),"%Y/%m/%d")
            Transaction_count = int(json_stock["data"][i][1].replace(",",""))
            Transaction_amount = int(json_stock["data"][i][2].replace(",",""))
            Opening_price = float(json_stock["data"][i][3])
            Highest_price = float(json_stock["data"][i][4])
            Lowest_price = float(json_stock["data"][i][5])
            Closing_price = float(json_stock["data"][i][6])

            if json_stock["data"][i][7] == "X0.00":
                Price_spread = 0.00
            else:
                Price_spread = float(json_stock["data"][i][7])

            Number_of_transactions = int(json_stock["data"][i][8].replace(",",""))

            dict_result = {
                "Number":str(stock),
                "Date":Date,
                "Transaction_count":Transaction_count,
                "Transaction_amount":Transaction_amount,
                "Opening_price":Opening_price,
                "Highest_price":Highest_price,
                "Lowest_price":Lowest_price,
                "Closing_price":Closing_price,
                "Price_spread":Price_spread,
                "Number_of_transactions":Number_of_transactions

            }

            result.append(dict_result)
            
        except ValueError :
            pass 
        
    return result


In [8]:
user = "root"
passwd = "Pn123456"
ip = "192.168.22.110:3306"
db_name = "Stock"
table_name = "Stock_data"

def List_to_mysql(user,passwd,ip,db_name,table_name,result_list):

    engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
    con = engine.connect()
    
    for item in result_list:
        df = pd.DataFrame(item, index=[0])
        try:
            df.to_sql(name=table_name,con=con,if_exists='append',index=False)

        except Exception as e:
            if 'PRIMARY' in str(e):
                pass

    con.close() 
    engine.dispose()

In [9]:
# '''
# CREATE TABLE IF NOT EXISTS `Stock_data` (
#     `Number` CHAR(20) NOT NULL,
#     `Date` DATE NOT NULL,
#     `Transaction_count` INT NOT NULL,
#     `Transaction_amount` INT NOT NULL,
#     `Opening_price` FLOAT NOT NULL,
#     `Highest_price` FLOAT NOT NULL,
#     `Lowest_price` FLOAT NOT NULL,
#     `Closing_price` FLOAT NOT NULL,
#     `Price_spread` CHAR(10) NULL DEFAULT NULL,
#     `Number_of_transactions` INT NOT NULL,
#     PRIMARY KEY (`Number`, `Date`)
# )
# COLLATE='latin1_swedish_ci';

# '''

In [10]:
# count = 0
# for stock in stock_list[12:]:
#     for m in range(1,7):
#         try :
#             local_date = "20200"+str(m)+"01"
#             timestamp = str(int(time.time()))
#             url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp
#             result_list = Get_stock_data(url)
#             List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)
#             count += 1
#             print("股票代號: ",stock,"月份: ",local_date,"請求次數: ",count)
#             time.sleep(3)
#         except:
#             print("股票代號: ",stock,"月份: ",local_date,"請求次數: ",count," timeout")
        


In [11]:
# if res.status_code == requests.codes.ok:
#     print("OK")

In [12]:
# stock = "9924"
# local_date = "20200601"
# timestamp = str(int(time.time()))
# url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp
# result_list = Get_stock_data(url)
# List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)

In [18]:
error_stock = []
error_date = []
count = 0
for stock in stock_list[-100:]:
    try :
        local_date = datetime.now().strftime('%Y%m%d')
        timestamp = str(int(time.time()))
        url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+local_date+"&stockNo="+stock+"&_=" +timestamp
        result_list = Get_stock_data(url)
        List_to_mysql(user=user,passwd=passwd,ip=ip,db_name=db_name,table_name=table_name,result_list=result_list)
        count += 1
        print("股票代號: ",stock," 請求次數: ",count)
        time.sleep(3)
    except:
        
        error_stock.append(stock)
        error_date.append(local_date)
        print("股票代號: ",stock," 請求次數: ",count,"  timeout")
        
error_dict = {"股票代號":error_stock,"日期":error_date}
df = pd.DataFrame(error_dict)



status_code:  200
股票代號:  6670  請求次數:  1
status_code:  200
股票代號:  6671  請求次數:  2
status_code:  200
股票代號:  6672  請求次數:  3
status_code:  200
股票代號:  6674  請求次數:  4
status_code:  200
股票代號:  6698  請求次數:  5
status_code:  200
股票代號:  6706  請求次數:  6
股票代號:  6715  請求次數:  6   timeout
status_code:  200
股票代號:  8011  請求次數:  7
status_code:  200
股票代號:  8016  請求次數:  8
status_code:  200
股票代號:  8021  請求次數:  9
股票代號:  8028  請求次數:  9   timeout
status_code:  200
股票代號:  8033  請求次數:  10
status_code:  200
股票代號:  8039  請求次數:  11
status_code:  200
股票代號:  8046  請求次數:  12
status_code:  200
股票代號:  8070  請求次數:  13
股票代號:  8072  請求次數:  13   timeout
status_code:  200
股票代號:  8081  請求次數:  14
股票代號:  8101  請求次數:  14   timeout
status_code:  200
股票代號:  8103  請求次數:  15
status_code:  200
股票代號:  8104  請求次數:  16
status_code:  200
股票代號:  8105  請求次數:  17
status_code:  200
股票代號:  8110  請求次數:  18
股票代號:  8112  請求次數:  18   timeout
status_code:  200
股票代號:  8114  請求次數:  19
status_code:  200
股票代號:  8131  請求次數:  20
status_code:  200
股票代號:  8

In [19]:
df

,股票代號,日期
0,6715,20200621
1,8028,20200621
2,8072,20200621
3,8101,20200621
4,8112,20200621
5,8271,20200621
6,8443,20200621
7,8996,20200621
8,9911,20200621
9,9926,20200621
